In [1]:
from numpy.random import seed


from sklearn.utils import shuffle

import numpy as np
from tensorflow.keras.metrics import Precision, Recall, Accuracy
import argparse
import os
from pathlib import Path


seed(1)
import tensorflow as tf
tf.random.set_seed(2)

import sys

sys.path.append("../src")

from deep_learning_models import *
from deep_learning_functions import load_prepare_data_eval
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from utils import set_allow_gpu_memory_growth


SEED = 23

In [2]:
data_dir = Path("../Daten/data_to_use")
assert data_dir.exists(), f"{data_dir} does not exist!"

dataset = "all"


In [3]:
if dataset == "all":
    X_static, X_recurrent, y = load_prepare_data_eval(data_dir, dataset)
else:
    X_static, X_recurrent, y = load_prepare_data_eval(data_dir, dataset, return_industry = False)

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/ibemmene/Documents/HSLU/MLCartel/ml-cartel-detection/notebooks/../src/deep_learning_functions.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_static['industry'].replace(datanames,[0, 1, 2, 3, 4], inplace=True)


In [4]:
models = [
        ('BiLSTM_Dense',BiLSTMDenseModel), # lambda:BiLSTMDenseModel()
        ('BiLSTM_Dense2', BiLSTMDenseModel_2),
        ('BiLSTM_doubleDense2', BiLSTM_doubleDenseModel_2),
        ('BiLSTM_LSTM_Dense', BiLSTM_LSTMDenseModel),
        ('BiLSTM_LSTM_doubleDense', BiLSTM_LSTM_doubleDenseModel),
        ##('BiLSTM_LSTM', BiLSTM_LSTMModel),
        ('BiLSTM',BiLSTMModel)
    ]

In [5]:
model_name, model_factory = models[0]

In [6]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

In [7]:
for i,[train_index, test_index] in enumerate(skf.split(X_static, y)):
    scaler = StandardScaler()

    train_index = shuffle(train_index)
    test_index = shuffle(test_index)
    
    X_static_train = scaler.fit_transform(X_static[train_index])
    X_static_test = scaler.transform(X_static[test_index])
    
    model = model_factory()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[Precision(), Recall(), Accuracy()])

Metal device set to: Apple M1


2023-11-22 13:55:26.275505: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-22 13:55:26.275532: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
hist = model.fit([X_recurrent[train_index], X_static_train],
                             y[train_index],
                             validation_data=([X_recurrent[test_index], X_static_test], y[test_index])
                             )

2023-11-22 13:55:29.803881: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-22 13:55:31.716883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-22 13:55:32.134514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-22 13:55:32.146987: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<32x609x1

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_16' defined at (most recent call last):
    File "/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/

In [13]:
print(X_recurrent[train_index].shape)
print(X_static_train.shape)
print(y[train_index].shape)

(2833, 609, 1)
(2833, 10)
(2833,)
